# Gramática Libre de Contexto
## Lectura de archivos

In [235]:
file = open(r"C:\Users\hrosa\OneDrive\Documentos\CIC IPN\Primer Semestre\Teoria Computacion\GLC\GLC.txt")

In [236]:
content = file.read()

In [237]:
content.split('\n')

['S -> AB', 'A -> E', 'B -> C', 'C -> D', 'D -> b', 'E -> a']

In [238]:
# split_content = content.split('\n')
# stack = []
# for i in split_content:
#     rules = []
#     for j in i.split():
#         rules.append([j])
#     stack.append(rules)
#     # print(i)

# print(f'stack: {stack}')

split_content = content.split('\n')
stack = []
for r in split_content:
    rules = []
    for s in r.split():
        rules.append([s])
    stack.append(rules)


In [239]:
# rules = []
# for i in stack:
#     print(i)
#     rules.append([i.split()])
# print(f'rules:{rules}')

In [240]:
# print(f'{rules[0]},\n {rules[1]}')

In [241]:
#se crean los diccionarios de cada regla de inferencia
rules = {}
for i in stack:
    head = i[0][0]
    productions = []
    for j in i[2:]:
        if j[0] != '|':
            productions.append(j[0])
            print(productions)
    rules[head] = productions
print(f'rules{rules}')

['AB']
['E']
['C']
['D']
['b']
['a']
rules{'S': ['AB'], 'A': ['E'], 'B': ['C'], 'C': ['D'], 'D': ['b'], 'E': ['a']}


## Simplificación de unificación no recursiva

In [89]:
#simplificacion de unificacion no recursiva
for k,v in rules.items():
    print(f'k:{k}')
    print(f'v:{v}')
    first = {}
    for i in v: #por cada elemento de producciones
        max_prefix = i #como no se tiene compun, se toma todo el elemento
        print(f'HOLA I{i}')
        for j in v: #por cada aparición de elemento: comparar contra los demás
            if j != i: #si esa comparación no es igual
                print(f'j:{j} != i:{i}')
                prefix_common = "" #se inicializa en cero, porque aún no hay prefijo comúb
                compare = 1 #caracter en común en comparación 
                #mientras no tenga longitud mayor y haya comun
                #compare 1: a == a
                #compare 2: ab != B
                while compare <= len(i) and compare <= len(j) and i[:compare] == j[:compare]:
                    prefix_common = i[:compare]
                    compare += 1

                counter = 0
                for production in v: #cuenta # de elementos con prefijo compun
                    if production.startswith(prefix_common):
                        counter += 1
                if prefix_common and counter > 1: #actualizar el término más común
                    max_prefix = prefix_common

        prefix_found = max_prefix
        body = i[len(prefix_found):] #no incluir el prefijo común

        if prefix_found not in first:
            first[prefix_found] = []
        first[prefix_found].append(body)
    print(first)

print(f'new rules: {first}')

k:A
v:['abB', 'aB', 'cdg', 'cdeB', 'cdfB']
HOLA IabB
j:aB != i:abB
j:cdg != i:abB
j:cdeB != i:abB
j:cdfB != i:abB
HOLA IaB
j:abB != i:aB
j:cdg != i:aB
j:cdeB != i:aB
j:cdfB != i:aB
HOLA Icdg
j:abB != i:cdg
j:aB != i:cdg
j:cdeB != i:cdg
j:cdfB != i:cdg
HOLA IcdeB
j:abB != i:cdeB
j:aB != i:cdeB
j:cdg != i:cdeB
j:cdfB != i:cdeB
HOLA IcdfB
j:abB != i:cdfB
j:aB != i:cdfB
j:cdg != i:cdfB
j:cdeB != i:cdfB
{'a': ['bB', 'B'], 'cd': ['g', 'eB', 'fB']}
new rules: {'a': ['bB', 'B'], 'cd': ['g', 'eB', 'fB']}


In [92]:
new_rules = {}
counter_r = 1
print(rules)
origin_rules = None
for k, v in rules.items():
    origin_rules = k

productions_origin = []

for k,v in first.items():
    if len(v) == 1: #si solo hay una producción:
        new_rules[k] = v
        productions_origin.append(k+v[0])
    else:
        aux = f"R{counter_r}"
        counter_r += 1
        new_rules[aux] = v
        productions_origin.append(k+aux)

rules_unified = {origin_rules:productions_origin}
for k,v in new_rules.items():
    rules_unified[k] = v

print(f'new rules: {rules_unified}')

{'A': ['abB', 'aB', 'cdg', 'cdeB', 'cdfB']}
new rules: {'A': ['aR1', 'cdR2'], 'R1': ['bB', 'B'], 'R2': ['g', 'eB', 'fB']}


In [91]:
for k,v in rules_unified.items():
    print(f'{k}->{"|".join(v)}')


A->aR1|cdR2
R1->bB|B
R2->g|eB|fB


## Simplicidad de símbolos redundantes
1. variables no productivas.
2. variables inalcanzables.

In [215]:
stack

[[['S'], ['->'], ['AB']],
 [['A'], ['->'], ['E']],
 [['B'], ['->'], ['C']],
 [['C'], ['->'], ['D']],
 [['D'], ['->'], ['b']],
 [['E'], ['->'], ['a']]]

In [216]:
rules

{'S': ['AB'], 'A': ['E'], 'B': ['C'], 'C': ['D'], 'D': ['b'], 'E': ['a']}

In [217]:
not_productive = []
for k,v in rules.items():
    print(f'k:{k} and v:{v}')
    productive = False

    for production in v:
        terminal_count = 0
        no_terminal_count = False

        for char in production:
            if char.islower(): #es terminal?
                terminal_count += 1
            if char.isupper():
                no_terminal_count = True

        if terminal_count >= 1 and not no_terminal_count:
            productive = True
            
    if not productive and k != 'S':
        not_productive.append(k)
print(not_productive)

k:S and v:['AB']
k:A and v:['E']
k:B and v:['C']
k:C and v:['D']
k:D and v:['b']
k:E and v:['a']
['A', 'B', 'C']


In [218]:
filter_no_productive = {}
for k,v in rules.items():
    if k not in not_productive:
        new_rules = []
        for production in v:
            productive = True
            for char in production:
                if char in not_productive:
                    print(char)
                    productive = False
            if productive:
                new_rules.append(production)
        if new_rules:
            filter_no_productive[k] = new_rules
print(filter_no_productive)
        

A
B
{'D': ['b'], 'E': ['a']}


In [224]:
filter_no_productive

{'D': ['b'], 'E': ['a']}

In [222]:
rule_s = filter_no_productive['S']
not_redundants = {}
inalcanzable = None
for k, v in filter_no_productive.items():
    for production in v:
        # print(production)
        # print(f'k:{k}')
        for i in rule_s:          
            # print(i)
            # print(f'---{k}')
            if k not in i and k != 'S':
                inalcanzable = k
print(inalcanzable)


for k,v in filter_no_productive.items():
    if k != inalcanzable:
        not_redundants[k] = v
print(not_redundants)

KeyError: 'S'

In [225]:
filter_no_productive['S']

KeyError: 'S'

## Simplicidad por reglas unitarias

In [242]:
rules

{'S': ['AB'], 'A': ['E'], 'B': ['C'], 'C': ['D'], 'D': ['b'], 'E': ['a']}

In [253]:
is_terminal = {}
for k,v in rules.items():
    for production in v:
        if production.islower():
            is_terminal[k] = production
            print(is_terminal)
    if is_terminal in v:
        print('yes')
print(is_terminal)

{'D': 'b'}
{'D': 'b', 'E': 'a'}
{'D': 'b', 'E': 'a'}


In [254]:
is_terminal

{'D': 'b', 'E': 'a'}